In [ ]:
### 1. initial setup (import package)###

# apply word2vec analysis
from gensim.models import word2vec
from gensim import models
from sklearn.metrics.pairwise import cosine_similarity
# support package
import time
import better_exceptions
import warnings
from itertools import combinations
from collections import Counter

# self-package
import question_module
import news_module

Building prefix dict from /home/skydome20/Desktop/dbot/dbot.py3/jieba_dict/dict.txt.big ...
Dumping model to file cache /tmp/jieba.u300776cf1c5f564361e07846b411a882.cache
Loading model cost 5.612 seconds.
Prefix dict has been built succesfully.
Building prefix dict from /home/skydome20/Desktop/dbot/dbot.py3/jieba_dict/dict.txt.big ...
2017-09-06 22:23:08,205 : DEBUG : Building prefix dict from /home/skydome20/Desktop/dbot/dbot.py3/jieba_dict/dict.txt.big ...
Loading model from cache /tmp/jieba.u300776cf1c5f564361e07846b411a882.cache
2017-09-06 22:23:08,222 : DEBUG : Loading model from cache /tmp/jieba.u300776cf1c5f564361e07846b411a882.cache
Loading model cost 5.337 seconds.
2017-09-06 22:23:13,559 : DEBUG : Loading model cost 5.337 seconds.
Prefix dict has been built succesfully.
2017-09-06 22:23:13,570 : DEBUG : Prefix dict has been built succesfully.


In [ ]:
### 2. initial setup (basic setup)###

# prepare wiki word2vec matrix
path = "/home/skydome20/Desktop/dbot/dbot.py3/wiki_word2vec_set/wiki_article_word2vector_200_skipgram5_model.bin"
wiki_model = models.Word2Vec.load_word2vec_format(path, binary=True)

# give question #
question = "金正恩和川普"
q_keywords = question_module.handler(question)
print(q_keywords)

In [4]:
### 3. parse related articles by given question ###
""" 
news_bigTable_path = news_module.news_parse(q_keywords, "+")

# the reference path
title_path = news_bigTable_path[0] 
news_path = news_bigTable_path[1]  
href_path = news_bigTable_path[2] 
"""
title_path = "/home/skydome20/Desktop/news_folder/title/台大_論文造假_title.txt"
news_path = "/home/skydome20/Desktop/news_folder/all/台大_論文造假.txt"
href_path = "/home/skydome20/Desktop/news_folder/href/台大_論文造假_href.txt"


In [5]:
### 4. Cut articles ###
cut_news_path = news_module.news_cut(news_path)

In [6]:
### 5. remove high similarity articles ### 
filter_news_path = news_module.news_similar_filter(cut_news_path)

2017-04-06 04:52:39,031 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-04-06 04:52:39,162 : INFO : built Dictionary(8125 unique tokens: ['日前', '化所前', '視', '第二次', '續任']...) from 178 documents (total 50825 corpus positions)


The original news: 178


2017-04-06 04:52:39,271 : INFO : collecting document frequencies
2017-04-06 04:52:39,283 : INFO : PROGRESS: processing document #0
2017-04-06 04:52:39,300 : INFO : calculating IDF weights for 178 documents and 8124 features (34217 matrix non-zeros)
2017-04-06 04:52:39,327 : INFO : using serial LSI version on this node
2017-04-06 04:52:39,334 : INFO : updating model with new documents
2017-04-06 04:52:39,450 : INFO : preparing a new chunk of documents
2017-04-06 04:52:39,475 : INFO : using 100 extra samples and 2 power iterations
2017-04-06 04:52:39,483 : INFO : 1st phase: constructing (8125, 110) action matrix
2017-04-06 04:52:39,508 : INFO : orthonormalizing (8125, 110) action matrix
2017-04-06 04:52:39,928 : INFO : 2nd phase: running dense svd on (110, 178) matrix
2017-04-06 04:52:40,011 : INFO : computing the final decomposition
2017-04-06 04:52:40,017 : INFO : keeping 10 factors (discarding 75.572% of energy spectrum)
2017-04-06 04:52:40,025 : INFO : processed documents up to #178


The keep news: 23
The keep news = [0, 2, 5, 6, 7, 8, 9, 10, 11, 14, 15, 18, 21, 27, 31, 33, 40, 44, 45, 49, 51, 85, 111]


In [11]:
### 6. associate keywords ### 
ass_keywords = news_module.news_associated_keywords(q_keywords, filter_news_path)

The number of news is :178
[('學術', 67), ('倫理', 46), ('調查', 39), ('研究', 38), ('造假', 29), ('台灣', 26), ('校長', 24), ('楊泮池', 22), ('科技部', 22), ('教授', 22), ('教育部', 19), ('論文', 17), ('作者', 17), ('掛名', 15)]


In [12]:
print(ass_keywords)

['學術', '倫理', '調查', '研究', '造假']


In [55]:
### 7. deduction on associated_keywords  ###
warnings.filterwarnings("ignore", category=DeprecationWarning) 

deduction_words = []

# check keywords in wiki_word2vec
for word in ass_keywords:
    try:
        wiki_model[word]
    except Exception as e:
        ass_keywords.remove(word)

print(ass_keywords)

# use combination of ass_keywords to apply deduction on keywords
for tuple_words in combinations(ass_keywords, 2):
    
    # word2vec addition result to deduction 
    combin_vec = wiki_model[tuple_words[0]] + wiki_model[tuple_words[1]]

    # cosine similarity for most related words for word2vec addition result
    lst = list()
    for voc in wiki_model.vocab:
        voc_vec = wiki_model[voc]
        cos_sim = cosine_similarity(combin_vec, voc_vec)
        lst.append( (voc, float(cos_sim)) )
        
    # top k most words 
    result = sorted(lst, key=lambda x: x[1], reverse=True)
    
    # store top k most words
    deduction_words.extend([tup_result[0] for tup_result in result[2:10]])
    
#print (deduction_words)

2017-04-06 13:41:37,758 : WARNING : direct access to vocab will not be supported in future gensim releases, please use model.wv.vocab


['學術', '倫理', '調查', '研究', '造假']


2017-04-06 13:43:42,842 : WARNING : direct access to vocab will not be supported in future gensim releases, please use model.wv.vocab
2017-04-06 13:45:34,067 : WARNING : direct access to vocab will not be supported in future gensim releases, please use model.wv.vocab
2017-04-06 13:47:31,517 : WARNING : direct access to vocab will not be supported in future gensim releases, please use model.wv.vocab
2017-04-06 13:49:32,446 : WARNING : direct access to vocab will not be supported in future gensim releases, please use model.wv.vocab
2017-04-06 13:51:29,816 : WARNING : direct access to vocab will not be supported in future gensim releases, please use model.wv.vocab
2017-04-06 13:53:26,990 : WARNING : direct access to vocab will not be supported in future gensim releases, please use model.wv.vocab
2017-04-06 13:55:15,169 : WARNING : direct access to vocab will not be supported in future gensim releases, please use model.wv.vocab
2017-04-06 13:57:04,751 : WARNING : direct access to vocab wil

In [56]:
### 8. top 5 deduction words ###
[word[0] for word in Counter(deduction_words).most_common(5)]

['商週', '山西疫苗', '論文造假', '媒體藝術史', '應用倫理學']

[356, 9, 2]
